In [ ]:
df = spark.read.csv("/FileStore/tables/city_temperature.csv", header=True, inferSchema=True)
df2 = spark.read.csv("/FileStore/tables/country_list.csv", header=True, inferSchema=True)


In [ ]:
df = df.filter(df.AvgTemperature != -99)


In [ ]:
# Create a temporary view
df.createOrReplaceTempView("city_temperature")
df2.createOrReplaceTempView("country_list")


In [ ]:
# Query 1
avg_temp_per_region = spark.sql("""
SELECT Region, AVG(AvgTemperature) as AverageTemperature
FROM city_temperature
GROUP BY Region
""")
avg_temp_per_region.show()

# Query 2
avg_temp_by_year_country_asia = spark.sql("""
SELECT Country, Year, AVG(AvgTemperature) as AverageTemperature
FROM city_temperature
WHERE Region = 'Asia'
GROUP BY Country, Year
ORDER BY Country, Year
""")
avg_temp_by_year_country_asia.show()


# Query 3
avg_temp_by_city_spain = spark.sql("""
SELECT City, AVG(AvgTemperature) as AverageTemperature
FROM city_temperature
WHERE Country = 'Spain'
GROUP BY City
""")
avg_temp_by_city_spain.show()


+--------------------+------------------+
|              Region|AverageTemperature|
+--------------------+------------------+
|              Africa| 74.40260231125495|
|                Asia|  68.1097225987458|
|              Europe| 51.94717142841552|
|Australia/South P...| 62.30369323842191|
|       North America| 56.15019771858279|
|         Middle East| 73.84068255374054|
|South/Central Ame...| 72.20202379397276|
+--------------------+------------------+

+----------+----+------------------+
|   Country|Year|AverageTemperature|
+----------+----+------------------+
|Bangladesh|1995| 79.35017301038064|
|Bangladesh|1996| 78.76584507042256|
|Bangladesh|1997| 77.15481927710844|
|Bangladesh|1998| 79.28705882352948|
|Bangladesh|1999| 77.08787878787875|
|Bangladesh|2000|  79.0985507246377|
|Bangladesh|2001| 78.95656934306568|
|Bangladesh|2002| 78.62374429223748|
|Bangladesh|2003| 78.82487562189054|
|Bangladesh|2004| 78.91494845360823|
|Bangladesh|2005| 77.85701754385963|
|Bangladesh|2006|  

In [ ]:
avg_temp_capitals = spark.sql("""
SELECT 
    ct.Country, 
    cl.capital AS Capital, 
    AVG(ct.AvgTemperature) as AverageTemperature
FROM 
    city_temperature AS ct
JOIN 
    country_list AS cl 
ON 
    ct.Country = cl.country AND ct.City = cl.capital
WHERE 
    cl.type = 'countryCapital'
GROUP BY 
    ct.Country, cl.capital
""")
avg_temp_capitals.show()


+--------------------+------------+------------------+
|             Country|     Capital|AverageTemperature|
+--------------------+------------+------------------+
|             Namibia|    Windhoek| 69.32590835454769|
|               China|     Beijing|   55.202587420158|
|          Mauritania|  Nouakchott| 78.11336918000707|
|        Sierra Leone|    Freetown| 81.71703655352484|
|             Tunisia|       Tunis| 67.08807409012111|
|Central African R...|      Bangui| 78.82184718529616|
|          Madagascar|Antananarivo| 65.45140391128601|
|            Ethiopia| Addis Ababa| 62.95038499230032|
|              Uganda|     Kampala| 72.58953292793392|
|       Guinea-Bissau|      Bissau| 82.33553368075677|
|               Egypt|       Cairo| 72.67604855315933|
|         North Korea|   Pyongyang|51.873763964163494|
|             Algeria|     Algiers| 64.37253818654507|
|             Morocco|       Rabat| 63.38186152345862|
|             Senegal|       Dakar| 76.33087262872628|
|         

In [ ]:
# Filter out rows where AvgTemperature is -99
df_temperature_filtered = df.filter(df.AvgTemperature != -99)
from pyspark.sql.functions import broadcast

# Broadcast the country_list DataFrame
df_country_list_broadcast = broadcast(df2)
from pyspark.sql.functions import col

# Join the DataFrames with aliases
df_joined = df_temperature_filtered.alias("temp").join(
    df_country_list_broadcast.alias("country"),
    (col("temp.Country") == col("country.country")) &
    (col("temp.City") == col("country.capital")) &
    (col("country.type") == "countryCapital")
)
from pyspark.sql import functions as F

# Compute the average temperature for each capital using alias
df_avg_temp = df_joined.groupBy("country.country", "country.capital").agg(F.avg("temp.AvgTemperature").alias("AverageTemperature"))
df_avg_temp.show()


+--------------------+------------+------------------+
|             country|     capital|AverageTemperature|
+--------------------+------------+------------------+
|             Namibia|    Windhoek| 69.32590835454769|
|               China|     Beijing|   55.202587420158|
|          Mauritania|  Nouakchott| 78.11336918000707|
|        Sierra Leone|    Freetown| 81.71703655352484|
|             Tunisia|       Tunis| 67.08807409012111|
|Central African R...|      Bangui| 78.82184718529616|
|          Madagascar|Antananarivo| 65.45140391128601|
|            Ethiopia| Addis Ababa| 62.95038499230032|
|              Uganda|     Kampala| 72.58953292793392|
|       Guinea-Bissau|      Bissau| 82.33553368075677|
|               Egypt|       Cairo| 72.67604855315933|
|         North Korea|   Pyongyang|51.873763964163494|
|             Algeria|     Algiers| 64.37253818654507|
|             Morocco|       Rabat| 63.38186152345862|
|             Senegal|       Dakar| 76.33087262872628|
|         

In [ ]:
df_temperature_filtered = df
df_country_list = df2
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

def filter_years(year):
    return year >= 2000

filter_years_udf = udf(filter_years, BooleanType())

df_temperature_filtered = df_temperature_filtered.filter(filter_years_udf(df_temperature_filtered.Year))
from pyspark.sql.functions import broadcast

df_country_list_broadcast = broadcast(df_country_list)
from pyspark.sql.functions import col

df_joined = df_temperature_filtered.alias("temp").join(
    df_country_list_broadcast.alias("country"),
    (col("temp.Country") == col("country.country")) &
    (col("temp.City") == col("country.capital")) &
    (col("country.type") == "countryCapital")
)
from pyspark.sql import functions as F

df_avg_temp = df_joined.groupBy("country.country", "country.capital").agg(F.avg("temp.AvgTemperature").alias("AverageTemperature"))
def format_output(capital, country, avg_temp):
    return f"{capital} is the capital of {country} and its average temperature is {avg_temp:.2f}"

format_output_udf = udf(format_output)

df_final_output = df_avg_temp.withColumn("FormattedOutput", format_output_udf(col("country.capital"), col("country.country"), col("AverageTemperature")))
df_final_output.select("FormattedOutput").show(truncate=False)


+--------------------------------------------------------------------------------------+
|FormattedOutput                                                                       |
+--------------------------------------------------------------------------------------+
|Windhoek is the capital of Namibia and its average temperature is 69.40               |
|Beijing is the capital of China and its average temperature is 55.14                  |
|Nouakchott is the capital of Mauritania and its average temperature is 78.07          |
|Freetown is the capital of Sierra Leone and its average temperature is 81.70          |
|Tunis is the capital of Tunisia and its average temperature is 67.17                  |
|Bangui is the capital of Central African Republic and its average temperature is 78.87|
|Antananarivo is the capital of Madagascar and its average temperature is 65.35        |
|Addis Ababa is the capital of Ethiopia and its average temperature is 63.14           |
|Kampala is the capit